In [1]:
import yaml
import db
from datetime import datetime, timedelta

In [2]:
# Carregar configurações do arquivo YAML
with open("config.yml", "r") as config_file:
    config = yaml.safe_load(config_file)

# Obter configurações do SQLite
db_config = config["sqlite"]
database_path = db_config["database_path"]
timeout = db_config.get("timeout", 5)  # Valor padrão de 5 segundos

In [59]:
data_inicial = '2020-01-01'
data_final = '2020-12-31'

#procurar a data de pregão mais próxima.
consulta_sql = f"""SELECT * FROM datas_pregao WHERE DATA BETWEEN ? AND ?"""
resultados = db.consultar_banco(database_path, consulta_sql, (data_inicial, data_final))

# Ajusta as datas inicial e final para o intervalo de datas encontrado
if resultados:
    data_inicial = resultados[0][0]
    data_final = resultados[-1][0]
    print(f"Data inicial: {data_inicial}")
    print(f"Data final: {data_final}")
else:
    print("Nenhum resultado encontrado.")




Data inicial: 2020-01-02
Data final: 2020-12-30


In [60]:
#procurar datas de vencimentos mais próximas.

def somar_dias(data_inicial, numero_dias):
    # Converte a string de entrada para um objeto de data
    data = datetime.strptime(data_inicial, "%Y-%m-%d")
    # Adiciona os dias à data
    nova_data = data + timedelta(days=numero_dias)
    # Retorna a nova data como string no mesmo formato
    return nova_data.strftime("%Y-%m-%d")

data_inicial_opcao = somar_dias(data_inicial, 30)

consulta_sql = f"""SELECT DATA FROM datas_vencimento WHERE DATA > ? LIMIT 1"""
resultado_consulta = db.consultar_banco(database_path, consulta_sql, [data_inicial_opcao])
data_inicial_opcao = resultado_consulta[0][0] if resultado_consulta else None
if data_inicial_opcao:
    print(f"Data de vencimento mais próxima: {data_inicial_opcao}")
else:
    print("Nenhuma data de vencimento encontrada após a data inicial.")


Data de vencimento mais próxima: 2020-02-17


In [61]:
def preco_spot_bova11(data_pregao):
    # Consulta o preço spot do BOVA11 para a data de pregão especificada
    consulta_sql = f"""SELECT PREULT FROM cotacoes_bova11 WHERE DTPREG = ?"""
    resultados = db.consultar_banco(database_path, consulta_sql, [data_pregao])
    preco_spot = resultados[0][0] if resultados else None
    return preco_spot

In [71]:
preco_spot = preco_spot_bova11(data_inicial)
strike_inicial = int(preco_spot + 5)
print(f"Preço spot do BOVA11: {preco_spot}")
print(f"Strike inicial: {strike_inicial}")

Preço spot do BOVA11: 114.24
Strike inicial: 119


In [78]:
# encontrar opcao mais próxima do strike
consulta_sql = f"""SELECT DISTINCT CODNEG, PREEXE FROM opcoes_bova11 WHERE DATVEN = ? AND PREEXE = ?"""

opcao_encontrada_codneg = False
opcao_encontrada_strike = False

for strike in range(strike_inicial, strike_inicial + 2):
    resultados = db.consultar_banco(database_path, consulta_sql, (data_inicial_opcao, strike))
    if resultados:
        opcao_encontrada_codneg = resultados[0][0]
        opcao_encontrada_strike = resultados[0][1]
        print(f"Opção encontrada {opcao_encontrada_codneg} Strike {opcao_encontrada_strike}")
        break




Opção encontrada BOVAB20 Strike 120.0


Primeira opção encontrada. Agora é montar a posição e fazer as rolagens necessárias
* Ideia é encerrar se chegar em 5 centavos e abrir uma nova ou rolar subindo o máximo de strikes strike


In [73]:
for resultado in resultados:
    print(f"Opção encontrada: {resultado}")

Opção encontrada: ('2020-01-02', 'BOVAB18', 118.0, '2020-02-17', 0.71, 0.71, 1.16, 1.14, 63600, '070')
Opção encontrada: ('2020-01-02', 'BOVAB20', 120.0, '2020-02-17', 0.4, 0.4, 0.62, 0.61, 35670, '070')
Opção encontrada: ('2020-01-03', 'BOVAB18', 118.0, '2020-02-17', 0.99, 0.99, 1.23, 1.21, 20340, '070')
Opção encontrada: ('2020-01-03', 'BOVAB20', 120.0, '2020-02-17', 0.48, 0.48, 0.7, 0.7, 37780, '070')
Opção encontrada: ('2020-01-06', 'BOVAB18', 118.0, '2020-02-17', 0.78, 0.72, 0.86, 0.76, 27800, '070')
Opção encontrada: ('2020-01-06', 'BOVAB20', 120.0, '2020-02-17', 0.42, 0.32, 0.45, 0.37, 24090, '070')
Opção encontrada: ('2020-01-07', 'BOVAB18', 118.0, '2020-02-17', 0.7, 0.56, 0.7, 0.66, 28960, '070')
Opção encontrada: ('2020-01-07', 'BOVAB20', 120.0, '2020-02-17', 0.29, 0.29, 0.35, 0.34, 6400, '070')
Opção encontrada: ('2020-01-08', 'BOVAB18', 118.0, '2020-02-17', 0.59, 0.56, 0.66, 0.56, 52400, '070')
Opção encontrada: ('2020-01-08', 'BOVAB20', 120.0, '2020-02-17', 0.34, 0.24, 0.3